In [14]:
import numpy as np
import pandas as pd

In [15]:
df = pd.read_csv("/home/seer/Documents/projects/freelance/freelance2/ml/data/raw/multi_repo_data.csv")

In [24]:
df.columns

Index(['repo_name', 'pr_number', 'html_url', 'state', 'is_draft', 'merged',
       'title', 'body', 'author_association', 'author_account_age_days',
       'additions', 'deletions', 'changed_files', 'commits_count', 'body_len',
       'created_day_of_week', 'created_hour', 'hours_to_merge',
       'review_comments_count', 'issue_comments_count',
       'requested_reviewers_count', 'labels', 'milestone', 'head_branch',
       'base_branch', 'file_extensions', 'is_risky'],
      dtype='str')

In [19]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 15303 entries, 0 to 15302
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   repo_name                  15303 non-null  str    
 1   pr_number                  15303 non-null  int64  
 2   html_url                   15303 non-null  str    
 3   state                      15303 non-null  str    
 4   is_draft                   15303 non-null  bool   
 5   merged                     15303 non-null  bool   
 6   title                      15303 non-null  str    
 7   body                       13745 non-null  str    
 8   author_association         15303 non-null  str    
 9   author_account_age_days    15303 non-null  int64  
 10  additions                  15303 non-null  int64  
 11  deletions                  15303 non-null  int64  
 12  changed_files              15303 non-null  int64  
 13  commits_count              15303 non-null  int64  
 14  b

In [17]:
df.describe()

,pr_number,author_account_age_days,additions,deletions,changed_files,commits_count,body_len,created_day_of_week,created_hour,hours_to_merge,review_comments_count,issue_comments_count,requested_reviewers_count
count,15303.000000,15303.000000,15303.000000,15303.000000,15303.000000,15303.000000,15303.000000,15303.000000,15303.000000,15303.000000,15303.000000,15303.000000,15303.000000
mean,24026.247729,3431.873227,275.308371,156.769914,7.753251,3.326341,1234.753970,2.524538,12.782200,144.523859,1.237535,1.563354,0.359015
std,42904.749978,1593.027909,3561.423951,3807.120929,51.478353,11.225235,3089.254562,1.856127,6.528028,584.929897,4.885993,2.916679,1.017221
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3496.000000,2173.000000,3.000000,1.000000,1.000000,1.000000,93.000000,1.000000,8.000000,1.105000,0.000000,0.000000,0.000000
50%,11536.000000,3479.000000,14.000000,4.000000,2.000000,1.000000,391.000000,2.000000,14.000000,12.100000,0.000000,1.000000,0.000000
75%,21881.000000,4825.000000,68.000000,22.000000,4.000000,3.000000,1145.500000,4.000000,18.000000,70.900000,0.000000,2.000000,0.000000
max,288593.000000,6533.000000,256344.000000,430034.000000,3287.000000,751.000000,65535.000000,6.000000,23.000000,21911.890000,200.000000,75.000000,20.000000


In [28]:
# ours_to_merge > 70 (75th percentile is 70.9)

# Captures PRs in the top 25% slowest to merge
# ~3 days, which is reasonable threshold for "delayed"

# review_comments_count >= 3

# 75% of PRs have 0 review comments
# Mean is only 1.24
# Having 3+ review comments means the PR needed significant discussion/changes
# This is a strong signal of complexity or issues

# issue_comments_count >= 5

# 50% have ≤1, 75% have ≤2
# Mean is 1.56
# 5+ comments suggests active discussion or concerns

In [18]:
# is_risky = (hours_to_merge > 70) | (review_comments_count >= 3) | (issue_comments_count >= 5)

df['is_risky'] = ((df['hours_to_merge'] > 70) | 
                  (df['review_comments_count'] >= 3) | 
                  (df['issue_comments_count'] >= 5)).astype(int)

print("Target Variable Distribution:")
print(df['is_risky'].value_counts())
print("\nPercentage:")
print(df['is_risky'].value_counts(normalize=True) * 100)

Target Variable Distribution:
is_risky
0    10013
1     5290
Name: count, dtype: int64

Percentage:
is_risky
0    65.431615
1    34.568385
Name: proportion, dtype: float64


In [20]:
# make a copy to preserve the original 

df_processed = df.copy()

In [21]:
(df_processed["body"].isna()).sum()
df_processed['body'] = df_processed['body'].fillna('')
df_processed['labels'] = df_processed['labels'].fillna('none')
df_processed['milestone'] = df_processed['milestone'].fillna('none')
df_processed['file_extensions'] = df_processed['file_extensions'].fillna('none')

In [22]:
# body len for body filled with empty string should be zero 
df_processed.loc[df_processed['body'] == '', 'body_len'] = 0

In [23]:
df_processed.info()

<class 'pandas.DataFrame'>
RangeIndex: 15303 entries, 0 to 15302
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   repo_name                  15303 non-null  str    
 1   pr_number                  15303 non-null  int64  
 2   html_url                   15303 non-null  str    
 3   state                      15303 non-null  str    
 4   is_draft                   15303 non-null  bool   
 5   merged                     15303 non-null  bool   
 6   title                      15303 non-null  str    
 7   body                       15303 non-null  str    
 8   author_association         15303 non-null  str    
 9   author_account_age_days    15303 non-null  int64  
 10  additions                  15303 non-null  int64  
 11  deletions                  15303 non-null  int64  
 12  changed_files              15303 non-null  int64  
 13  commits_count              15303 non-null  int64  
 14  b

## Feature engineering

In [25]:
df_processed['total_changes'] = df_processed['additions'] + df_processed['deletions']
df_processed['change_ratio'] = df_processed['additions'] / (df_processed['deletions'] + 1)
df_processed['avg_changes_per_file'] = df_processed['total_changes'] / (df_processed['changed_files'] + 1)
df_processed['commits_per_file'] = df_processed['commits_count'] / (df_processed['changed_files'] + 1)
df_processed['title_length'] = df_processed['title'].str.len()
df_processed['is_weekend'] = (df_processed['created_day_of_week'] >= 5).astype(int)
df_processed['is_business_hours'] = ((df_processed['created_hour'] >= 9) & 
                                      (df_processed['created_hour'] <= 17)).astype(int)

In [26]:
# Create binary features for presence/absence
df_processed['has_body'] = (df_processed['body'] != '').astype(int)
df_processed['has_labels'] = (df_processed['labels'] != 'none').astype(int)
df_processed['has_milestone'] = (df_processed['milestone'] != 'none').astype(int)
df_processed['has_file_extensions'] = (df_processed['file_extensions'] != 'none').astype(int)

In [27]:
print("Hello, world!")

Hello, world!


### encoding

In [28]:
from sklearn.preprocessing import LabelEncoder

In [29]:
le = LabelEncoder()
df_processed['author_association_encoded'] = le.fit_transform(df_processed['author_association'])

In [30]:
print("\nis_draft distribution:")
print(df_processed['is_draft'].value_counts())
print("\nmerged distribution:")
print(df_processed['merged'].value_counts())

# very small number of is_draft not gonna help 


is_draft distribution:
is_draft
False    15151
True       152
Name: count, dtype: int64

merged distribution:
merged
True    15303
Name: count, dtype: int64


In [31]:
# let's drop unnecessary columns
columns_to_drop = [
  'repo_name', # not needed
  'pr_number', # not needed
  'html_url', # not needed
  'state', # all are merged not useful 
  'merged', # again all same values not useful
  'title', # text, extracted title_length already 
  'body', # we have body_len & has_body, contents are not necessary to know if the pr is risky
  'author_association', # we already encoded it
  'labels', # we have has_labels
  'milestone', # we have has_milestone
  'head_branch', # too many unique values
  'base_branch', # same
  'file_extensions', # already have has_file_extensions
  'hours_to_merge',      # Used to create target
  'review_comments_count',  # Used to create target
  'issue_comments_count'    # Used to create target
]

df_processed = df_processed.drop(columns=columns_to_drop)

In [32]:
print(df_processed.columns.tolist())
print("\nShape:", df_processed.shape)
print(df_processed.dtypes.value_counts())

['is_draft', 'author_account_age_days', 'additions', 'deletions', 'changed_files', 'commits_count', 'body_len', 'created_day_of_week', 'created_hour', 'requested_reviewers_count', 'is_risky', 'total_changes', 'change_ratio', 'avg_changes_per_file', 'commits_per_file', 'title_length', 'is_weekend', 'is_business_hours', 'has_body', 'has_labels', 'has_milestone', 'has_file_extensions', 'author_association_encoded']

Shape: (15303, 23)
int64      19
float64     3
bool        1
Name: count, dtype: int64


In [33]:
df_processed["is_draft"] = df_processed["is_draft"].astype(int)

In [34]:
df_processed.info()

<class 'pandas.DataFrame'>
RangeIndex: 15303 entries, 0 to 15302
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   is_draft                    15303 non-null  int64  
 1   author_account_age_days     15303 non-null  int64  
 2   additions                   15303 non-null  int64  
 3   deletions                   15303 non-null  int64  
 4   changed_files               15303 non-null  int64  
 5   commits_count               15303 non-null  int64  
 6   body_len                    15303 non-null  int64  
 7   created_day_of_week         15303 non-null  int64  
 8   created_hour                15303 non-null  int64  
 9   requested_reviewers_count   15303 non-null  int64  
 10  is_risky                    15303 non-null  int64  
 11  total_changes               15303 non-null  int64  
 12  change_ratio                15303 non-null  float64
 13  avg_changes_per_file        15303 non-null

In [35]:
X = df_processed.drop('is_risky', axis=1)
y = df_processed["is_risky"]

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [37]:
print("Training set:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("y_train distribution:")
print(y_train.value_counts(normalize=True) * 100)

print("\nTest set:")
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("y_test distribution:")
y_test.value_counts(normalize=True) * 100

Training set:
X_train shape: (12242, 22)
y_train shape: (12242,)
y_train distribution:
is_risky
0    65.430485
1    34.569515
Name: proportion, dtype: float64

Test set:
X_test shape: (3061, 22)
y_test shape: (3061,)
y_test distribution:


is_risky
0    65.436132
1    34.563868
Name: proportion, dtype: float64

In [38]:
# Summary
print("\n6. X_train summary statistics:")
print(X_train.describe())

print(f"- {X_train.shape[0]} training samples")
print(f"- {X_test.shape[0]} test samples")
print(f"- {X_train.shape[1]} features")
print(f"- Balanced target: 65% not risky, 35% risky")


6. X_train summary statistics:
           is_draft  author_account_age_days      additions      deletions  \
count  12242.000000             12242.000000   12242.000000   12242.000000   
mean       0.010211              3429.637478     273.239830     165.559386   
std        0.100535              1588.991759    3639.658386    4236.600587   
min        0.000000                 0.000000       0.000000       0.000000   
25%        0.000000              2179.000000       3.000000       1.000000   
50%        0.000000              3465.500000      14.000000       4.000000   
75%        0.000000              4820.000000      67.000000      21.000000   
max        1.000000              6533.000000  256344.000000  430034.000000   

       changed_files  commits_count      body_len  created_day_of_week  \
count   12242.000000   12242.000000  12242.000000         12242.000000   
mean        7.811877       3.339079   1228.263601             2.537657   
std        54.328419      12.017896   2995.

In [39]:
df_processed.to_csv("/home/seer/Documents/projects/freelance/freelance2/ml/data/processed/processed_data.csv", index=False)